In [4]:
from Levenshtein import opcodes, distance

In [ ]:
import pandas as pd
from pandas import DataFrame
pd.get_option("display.max_columns")
pd.set_option("display.max_rows", None)


## 读取打轴给定文本，和语音识别文本
给定文本命名为label, 语音识别文本命名为hypo

In [5]:

label = open("/home/xushuli/git-repo/temporalfy/data/_4-hypOut.txt", "r").read() 
hypo = open("/home/xushuli/git-repo/temporalfy/data/_4-refOut.txt", "r").read()

## 找出 One-Shot 稀有字（字-索引元组）
经过两次过滤筛选出分段质量高的稀有字，筛选条件分别是“稀有字间距”和“稀有字邻域标签-预测子串”字错率。

In [7]:

def get_both_rare_word_index_pairs(hypo, label, distance_threhold=5, cer_threhold=0.5, neibor_radius=15):
    def get_rare_word(hypo, label):
        label_word_2_cnt = {}

        for char in label:
            cnt = label_word_2_cnt.get(char, 0)
            cnt += 1
            label_word_2_cnt[char] = cnt

        label_rares = set(map(lambda x:x[0], filter(lambda x:x[1] == 1,  label_word_2_cnt.items())) )

        hypo_word_2_cnt = {}

        for char in hypo:
            cnt = hypo_word_2_cnt.get(char, 0)
            cnt += 1
            hypo_word_2_cnt[char] = cnt

        hypo_rares = set(map(lambda x:x[0], filter(lambda x:x[1] == 1,  hypo_word_2_cnt.items())) )
        return label_rares.intersection(hypo_rares)

    def filter_by_word_distance(l, distance_threhold):
        assert len(l) > 1
        filtered_l = []
        for (w0, w0i), (w1, w1i) in zip(l[:-1], l[1:]):
            distance = w1i - w0i
            if distance > distance_threhold:
                filtered_l.append((w0, w0i))
        filtered_l.append((w1, w1i))
        return filtered_l

    def filter_by_neighbor_cer_between_hypo_label(l, hypo, label, cer_threhold, radius):
        def neibor(index, text, radius=5):
            left, right  = index - radius, index + radius
            left = max(0, left)
            right = min(len(text), right)
            return text[left:right ]

        def cer(hypo, label):
            return distance(hypo, label) / len(label)

        assert len(l) > 1

        filtered_l = []

        for w, li in l:
            hi = hypo.index(w)
            h_neibor, l_neibor = neibor(hi, hypo, radius), neibor(li, label, radius)
            if cer(h_neibor, l_neibor) > cer_threhold:
                continue
            else:
                filtered_l.append((w, li))
        return filtered_l

    rare_words = get_rare_word(hypo, label)
    l = [ (w, label.index(w) ) for w in rare_words]
    l = sorted(l, key=lambda x:x[1]) # before filter must sort

    l = filter_by_word_distance(l, distance_threhold=distance_threhold)
    l = filter_by_neighbor_cer_between_hypo_label(l, hypo, label, cer_threhold=cer_threhold,radius=neibor_radius)

    h = [ (w, hypo.index(w)) for w, _ in l ]

    return h,l


In [8]:
h,l = get_both_rare_word_index_pairs(hypo, label, distance_threhold=15)
h,l

([('街', 2628),
  ('渠', 2698),
  ('乱', 2731),
  ('友', 2918),
  ('尤', 3248),
  ('淆', 5076),
  ('美', 6274),
  ('演', 6372),
  ('惜', 7051),
  ('驳', 7067),
  ('远', 7238),
  ('填', 8785),
  ('佣', 8844),
  ('拐', 8992),
  ('爆', 9020),
  ('杂', 9812),
  ('默', 14874),
  ('赋', 15226),
  ('舍', 15578),
  ('楚', 17045),
  ('泥', 17087),
  ('甚', 21027),
  ('凌', 22034),
  ('劣', 23744),
  ('冲', 25852),
  ('战', 27606),
  ('板', 27983),
  ('疵', 28329),
  ('兑', 28994),
  ('摘', 29228),
  ('融', 31711),
  ('缘', 32393),
  ('励', 32551),
  ('摊', 34121),
  ('秘', 34170),
  ('浮', 35377),
  ('托', 35568),
  ('乐', 35945),
  ('卡', 36759),
  ('怨', 37723),
  ('宪', 38768),
  ('皆', 39031),
  ('疏', 39718),
  ('策', 39981),
  ('纯', 40086),
  ('庄', 40697),
  ('弄', 40733),
  ('耕', 41045),
  ('寒', 41854),
  ('怠', 44059),
  ('恤', 44324),
  ('搜', 44827),
  ('横', 45868),
  ('肆', 46000),
  ('触', 47133),
  ('似', 47160),
  ('映', 48191),
  ('逐', 48681),
  ('歉', 48758),
  ('捕', 48866),
  ('勾', 49859),
  ('套', 50382),
  ('滞', 51497),
  ('赡', 

## 计算空间复杂度

分别计算了拆分字串前，和拆分字串后的空间复杂度

In [20]:
def get_substr_lengths(word_index_pairs):
    indexs = [x[1] for x in word_index_pairs]
    def lengths(indexs):
        minus_pairs = zip([0, *indexs[:-1]], indexs[1:])
        return [i1-i0 for i0, i1 in minus_pairs]
    output = lengths(indexs)
    print("length of the max substr：",max(output))
    print("length of the min substr：",min(output))
    return output

spc_cmlx_splt = sum([ m*n  for m,n in zip(get_substr_lengths(h), get_substr_lengths(l))])

spc_cmlx_long = len(hypo) * len(label)

{"split": spc_cmlx_splt, "long": spc_cmlx_long, "reduction": spc_cmlx_splt / spc_cmlx_long}


length of the max substr： 5854
length of the min substr： 103
length of the max substr： 5857
length of the min substr： 104


{'split': 386941585, 'long': 6109134352, 'reduction': 0.06333820189652951}

In [22]:
def neibor(index, text, radius=5):
    left, right  = index - radius, index + radius
    left = max(0, left)
    right = min(len(text), right)
    return text[left:right ]

def cer(hypo, label):
    return distance(hypo, label) / len(label)


for (hw, hi), (lw, li) in zip(h,l):
    h_neibor, l_neibor = neibor(hi, hypo, 15), neibor(li, label, 15)
    print(hw, lw, hi, li , cer(h_neibor, l_neibor), h_neibor, l_neibor, sep=" || ")

街 || 街 || 2628 || 2648 || 0.03333333333333333 || 设立的基层人民调解组织三在乡镇街道设立的具有劳动争议调解职能 || 设立的基层人民调解组织在在乡镇街道设立的具有劳动争议调解职能
渠 || 渠 || 2698 || 2719 || 0.0 || 偿劳动者维权应当采用法律规定的渠道如协商调解劳动监察投诉申请 || 偿劳动者维权应当采用法律规定的渠道如协商调解劳动监察投诉申请
乱 || 乱 || 2731 || 2752 || 0.03333333333333333 || 裁法院起诉等单方采取组织罢工扰乱生产或者其他不当方式维权的不 || 裁法院起诉的单方采取组织罢工扰乱生产或者其他不当方式维权的不
友 || 友 || 2918 || 2944 || 0.0 || 工伤认定并提供工作服工作证或工友证言等证据证明劳动关系的成立 || 工伤认定并提供工作服工作证或工友证言等证据证明劳动关系的成立
尤 || 尤 || 3248 || 3274 || 0.06666666666666667 || 的律师为经济困难或特殊案件的人尤其是农村给予无偿提供法律服务 || 的历史为经济困难或特殊案件的人尤其是农村给予无偿提供法律服务
淆 || 淆 || 5076 || 5099 || 0.0 || 企业全体职工都具有约束力容易混淆的一点是虽然有时集体合同是由 || 企业全体职工都具有约束力容易混淆的一点是虽然有时集体合同是由
美 || 美 || 6274 || 6295 || 0.2 || 彩与形状图案的结合所作出的富有美感并适于工业应用的新设计发明 || 形状图案的结合所作出的具有富有美感并适用于工业应用的新设计发
演 || 演 || 6372 || 6390 || 0.0 || 同财产已经不受双方或一方掌控就演变成一方向另一方要求赔偿的债 || 同财产已经不受双方或一方掌控就演变成一方向另一方要求赔偿的债
惜 || 惜 || 7051 || 7080 || 0.0 || 给被告一个机会希望双方更能够珍惜夫妻之间的感情维系夫妻关系判 || 给被告一个机会希望双方更能够珍惜夫妻之间的感情维系夫妻关系判
驳 || 驳 || 7067 || 7096 || 0.0 || 夫妻之间的感情维系夫妻关系判决驳回起诉但是不能固化的认为第一 || 夫妻之

In [23]:
print( hypo[:h[1][1]] )
for (h0w, h0i), (h1w, h1i) in zip(h[:-1], h[1:]):
    print(h0w,h0i,h1i, hypo[h0i:h1i] )
print( hypo[h[-1][1]:] )

如果合同没有约定时间但是符合以下条件的就是有效的1订立合同时当事人双方都具有相应的民事行为能力2双方在订立合同时意思表示真实不存在欺诈胁迫等情形3合同的内容不违反法律行政法规的强制性规定不违背公序良俗满足以下条件的协议是有效的1订立协议时当事人双方都具有相应的民事行为能力2双方在订立协议时意思表示真实不存在欺诈胁迫等情形3协议的内容不违反法律行政法规的强制性规定不违背公序良俗合同解除后担保责任不一定能够免除如果主合同尚未履行就解除的则担保责任可以免除因为此时担保债权尚未发生所以不需要承担担保责任或者主合同是因债权人的责任而解除的担保责任也可以免除如果是中标后中标人不签合同的则可以取消其中标资格投标保证金不予退还然后由有关行政监督部门责令改正可以处中标项目金额5以上10以下的罚款如果是招标人拒绝签订合同的中标人可以书面催告其签订合同如果其还拒绝的则由有关行政监督部门责令改正可以处中标项目金额5以上10以下的罚款符合以下条件的借款合同就是有效的1订立合同时当事人双方都具有相应的民事行为能力2双方在订立合同时意思表示真实不存在欺诈胁迫等情形3合同的内容不违反法律行政法规的强制性规定不违背公序良俗父母离婚时子女无论是否成年都无权参与财产分割离婚时对于夫妻共同财产首先由离婚双方协商分割如果协商分割不成可以向法院起诉法院根据财产的具体情况按照照顾子女女方和无过错方权益的原则判决对于夫妻共同财产原则上是均等分割根据生产生活的实际需要和财产的来源等情况具体处理时也可以有所差别夫妻以书面形式约定婚姻关系存续期间所得的财产以及婚前财产归各自所有共同所有或部分各自所有部分共同所有的离婚时应按约定处理双方对夫妻共同财产中的房屋价值及归属无法达成协议时人民法院按以下情形分别处理一双方均主张房屋所有权并且同意竞价取得的应当准许二一方主张房屋所有权的由评估机构按市场价格对房屋作出评估取得房屋所有权的一方应当给予另一方相应的补偿三双方均不主张房屋所有权的根据当事人的申请拍卖变卖房屋就所得价款进行分割原则上夫妻一方婚前个人财产不因婚姻关系的延续而转化为夫妻共同财产但可以通过以下方式变成夫妻共同财产1赠与赠与是一种无偿转移财产的行为赠与的法律后果是财产所有权的转移赠与完成后不能把已经送出去的财产再要回来如果是房屋汽车的赠与应当办理登记手续2夫妻财产约定夫妻可以通过书面财产约定的方式约定婚前个人财产为夫

In [24]:
print( label[:l[1][1]] )
for (l0w, l0i), (l1w, l1i) in zip(l[:-1], l[1:]):
    print(l0w,l0i,l1i, label[l0i:l1i] )
print( label[l[-1][1]:] )

如果合同没有约定时间但是符合以下条件的就是有效的一订立合同时当事人双方都具有相应的民事行为能力二双方在订立合同时意思表示真实不存在欺诈胁迫等情形三合同的内容不违反法律行政法规强制性规定不违背公序良俗满足以下条件的协议是有效的一订立协议时当事人双方都具有相应的民事行为能力二双方在订立协议时意思表示真实不存在欺诈胁迫等情形三协议的内容不违反法律行政法规的强制性规定不违背公序良俗合同解除后担保责任不一定能够免除如果主合同尚未履行就解除则担保责任可以免除因为此时担保债权尚未发生所以不需要承担担保责任或者主合同是因侵权人的责任而解除的担保责任也可以免除如果是中标后中标人不签合同的则可以取消其其中标资格投标保证金不予退还然后由有关行政监督部门责令改正可以处中标项目金额百分之千分之五以上千分之十以下的罚款如果是招标人拒绝签订合同的中标人可以书面催告其签订合同如果其还拒绝的则由有关行政监督部门责令改正可以处中标项目金额千分之五以上千分之十以下的罚款符合以下条件的借款合同就是有效的一订立合同时当事人双方具有相应的民事行为能力二双方在订立合同时意思表示真实不存在欺诈胁迫等情形三合同的内容不违反法律行政法规的强制性规定不违背公序良俗父母离婚时子女无论是否成年都无权参与财产分割离婚时对于夫妻共同财产首先由离婚双方协商分割如果协商分割不成可以向法院起诉法院根据财产的具体情况按照照顾子女女方和无故作坊权益的原则判决对于夫妻共同财产原则上是均等分割根据生产生活的实际需要和财产的来源等情况具体处理时也可以有所差别夫妻以书面形式约定婚姻关系存续期间所得的财产以及婚前财产归各自所有共同所有或部分各自所有部分共同所有的离婚时应按约定处理双方对夫妻共同财产中的房屋价值及归属无法达成协议时人民法院按以下情形分别处理一双方君主让房屋所有权并且同意竞价取得的应当准许二一方主张房屋所有权的由评估机构按照市场价格对房屋作出评估取得房屋所有权的一方应当给予另一方相应的补偿在双方均不主张房屋所有权的根据当事人的申请拍卖变卖房屋就所得价款进行分割原则上夫妻一方婚前个人财产不因婚姻关系的延续而转化为夫妻共同财产但可以通过以下方式变成夫妻共同财产一赠与赠与是一种无偿转移财产的行为赠与的法律后果是财产所有权的转移赠与完成后不能把已经送出去的财产再要回来如果是房屋汽车的赠与应当办理登记手续二夫妻财产约定夫妻可以通过书面财产约定的

# 利用Levesthein的编辑错误信息进行对齐

In [21]:
def algin_text_pair(hypo, label):
    pass